In [27]:
import networkx as nx
import numpy as np

def get_adj(G:nx.Graph) -> np.matrix:
    return nx.to_numpy_array(G)

def gamma(A:np.matrix, i, j, q) -> np.float32:
    """
    Input:
    A: weighted or unweighted adjacency matrix of a directed graph
    i: row
    j: column
    q: parameter
    Returns:

    """
    return np.exp(1j * np.pi * q * (A[i, j] - A[j, i]))

def degree_matrix(A:np.matrix):
    return np.diag(np.sum(A, axis=1))

def hermitian_laplacian(G:nx.DiGraph, q) -> np.ndarray:
    assert isinstance(G, nx.DiGraph), "method is designed for directed graphs"
    A = get_adj(G)
    A_s = (A + A.T)/2
    D = degree_matrix(A_s)
    Gamma = np.zeros(A.shape, dtype=np.complex64)
    for i, j in zip(*A.nonzero()):
        Gamma[i, j] = gamma(A, i, j, q)

    Gamma[Gamma == 0] = 1

    L_q = D - np.multiply(Gamma,A_s)
    
    # if degree_normalize:
    #     inv_D = 
    #     L_q = 
    return L_q

In [28]:
G = nx.karate_club_graph().to_directed()
N = len(G.nodes)
q = 0.2
L_q = hermitian_laplacian(G, q)
L_q

array([[16.+0.j, -1.+0.j, -1.+0.j, ..., -1.+0.j,  0.+0.j,  0.+0.j],
       [-1.+0.j,  9.+0.j, -1.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
       [-1.+0.j, -1.+0.j, 10.+0.j, ...,  0.+0.j, -1.+0.j,  0.+0.j],
       ...,
       [-1.+0.j,  0.+0.j,  0.+0.j, ...,  6.+0.j, -1.+0.j, -1.+0.j],
       [ 0.+0.j,  0.+0.j, -1.+0.j, ..., -1.+0.j, 12.+0.j, -1.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j, ..., -1.+0.j, -1.+0.j, 17.+0.j]])

In [29]:
eigenvalues, U = np.linalg.eig(L_q)
U = np.matrix(U)
eigenvalues = np.real(eigenvalues) # by definition, these must be real
eigenvalues.shape, U.shape

((34,), (34, 34))

In [30]:
def low_pass_filter_kernel(eigs, c=1):
    return 1/(1+c*eigs)

s = 1 # scale parameter

G_hat_s = np.diag(low_pass_filter_kernel(eigenvalues*s))
G_hat_s.shape

(34, 34)

In [31]:
U_star = U.H
delta = np.eye(N)

In [32]:
psi = U @ G_hat_s @ U_star @ delta

In [33]:
psi

matrix([[0.09760604+0.j, 0.04053624+0.j, 0.03252644+0.j, ...,
         0.02486675+0.j, 0.01554225+0.j, 0.01690973+0.j],
        [0.04053624+0.j, 0.14560895+0.j, 0.040455  +0.j, ...,
         0.01708206+0.j, 0.01814567+0.j, 0.02012047+0.j],
        [0.03252644+0.j, 0.040455  +0.j, 0.12922497+0.j, ...,
         0.02429423+0.j, 0.02936881+0.j, 0.02653306+0.j],
        ...,
        [0.02486675+0.j, 0.01708206+0.j, 0.02429423+0.j, ...,
         0.18596729+0.j, 0.03569316+0.j, 0.03404643+0.j],
        [0.01554225+0.j, 0.01814567+0.j, 0.02936881+0.j, ...,
         0.03569316+0.j, 0.11916963+0.j, 0.04128168+0.j],
        [0.01690973+0.j, 0.02012047+0.j, 0.02653306+0.j, ...,
         0.03404643+0.j, 0.04128168+0.j, 0.09337072+0.j]])

In [35]:
np.all(np.isreal(psi))

True

In [36]:
psi = np.real(psi)

In [38]:
psi.shape # we now have the wavelets, lets compute features according to graphwave

(34, 34)

In [ ]:
def phi(psi, i, s, t):
    """S is needed as well"""
    N = psi.shape[0]
    return 1/N * np.sum(
        np.exp(1j * t)
    )